In [1]:
pip install pyspark


In [2]:
from google.colab import drive
drive.mount('/content/ColabFolder/wc2018-players')

Mounted at /content/ColabFolder/wc2018-players


In [3]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [4]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

from pyspark.sql.types import *

CRIAR / INICIAR SESSÃO PYSPARK

In [5]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('PySpark_01')
    .getOrCreate()
)

Criar DF / Ler arquivo

In [6]:
df = spark.read.csv('/content/ColabFolder/wc2018-players.csv', header=True, inferSchema=True)

Exibir DF

In [7]:
df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



Verificar Tipos de coluna

In [8]:
df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)



Verificando dados Nulos

In [9]:
df.toPandas().isna().sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of       Team      #   Pos.  FIFA Popular Name  Birth Date  Shirt Name   Club  \
0    False  False  False              False       False       False  False   
1    False  False  False              False       False       False  False   
2    False  False  False              False       False       False  False   
3    False  False  False              False       False       False  False   
4    False  False  False              False       False       False  False   
..     ...    ...    ...                ...         ...         ...    ...   
731  False  False  False              False       False       False  False   
732  False  False  False              False       False       False  False   
733  False  False  False              False       False       False  False   
734  False  False  False              False       False       False  False   
735  False  False  False              False       False       False  False   

 

In [10]:
# se o df for muito grande, não utilizar toPandas
# for coluna in df.columns:
#   print(coluna, df.filter(df[coluna].isNull()).count())

Renomear Colunas

In [11]:
df = df.withColumnRenamed('Team', 'Selecao').withColumnRenamed('#', 'Numero').withColumnRenamed('Pos.', 'Posicao')\
.withColumnRenamed('FIFA Popular Name', 'Nome_FIFA').withColumnRenamed('Birth Date', 'Nascimento')\
.withColumnRenamed('Shirt Name', 'Nome Camiseta').withColumnRenamed('Club', 'Time').withColumnRenamed('Height', 'Altura')\
.withColumnRenamed('Weight', 'Peso')

df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
only showing top 5 rows



In [12]:
for coluna in df.columns:
  print(coluna, df.filter(df[coluna].isNull()).count())

Selecao 0
Numero 0
Posicao 0
Nome_FIFA 0
Nascimento 0
Nome Camiseta 0
Time 0
Altura 0
Peso 0


Selecioinar Colunas

In [13]:
df.select('Selecao', 'Nome_fifa').show(5)

+---------+------------------+
|  Selecao|         Nome_fifa|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



In [14]:
df.select(col('Selecao'), col('numero')).show(5)

+---------+------+
|  Selecao|numero|
+---------+------+
|Argentina|     3|
|Argentina|    22|
|Argentina|    15|
|Argentina|    18|
|Argentina|    10|
+---------+------+
only showing top 5 rows



In [15]:
df.select(df['Selecao']).show(5)

+---------+
|  Selecao|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



Selecionar Colunas com Alias

In [16]:
df.select(df['Selecao'].alias('Time')).show(5)

+---------+
|     Time|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



In [17]:
df.select(col('Selecao').alias('Time')).show(5)

+---------+
|     Time|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



In [18]:
df.select('Selecao Nome_FIFA Altura'.split()).show(5)

+---------+------------------+------+
|  Selecao|         Nome_FIFA|Altura|
+---------+------------------+------+
|Argentina|TAGLIAFICO Nicolas|   169|
|Argentina|    PAVON Cristian|   169|
|Argentina|    LANZINI Manuel|   167|
|Argentina|    SALVIO Eduardo|   167|
|Argentina|      MESSI Lionel|   170|
+---------+------------------+------+
only showing top 5 rows



Organizar Select

In [19]:
df.select('Nome_Fifa', 'Peso', 'Altura').show(5)

+------------------+----+------+
|         Nome_Fifa|Peso|Altura|
+------------------+----+------+
|TAGLIAFICO Nicolas|  65|   169|
|    PAVON Cristian|  65|   169|
|    LANZINI Manuel|  66|   167|
|    SALVIO Eduardo|  69|   167|
|      MESSI Lionel|  72|   170|
+------------------+----+------+
only showing top 5 rows



Filtro

In [20]:
df.filter('Selecao = "Brazil"').show(11)

+-------+------+-------+-----------------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|        Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+-----------------+----------+-------------+--------------------+------+----+
| Brazil|    18|     MF|             FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|           TAISON|13.01.1988|       TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|      FERNANDINHO|04.05.1985|  FERNANDINHO|Manchester City F...|   179|  67|
| Brazil|    22|     DF|           FAGNER|11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    10|     FW|           NEYMAR|05.02.1992|    NEYMAR JR|Paris Saint-Germa...|   175|  68|
| Brazil|    11|     MF|PHILIPPE COUTINHO|12.06.1992|  P. COUTINHO|  FC Barcelona (ESP)|   172|  68|
| Brazil|     7|     FW|    DOUGLAS COSTA|14.09.1990|     D. COSTA|   Juventus FC (ITA)|   

In [21]:
df.filter(col('Nome Camiseta') == "FRED").show()

+-------+------+-------+---------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+---------+----------+-------------+--------------------+------+----+
| Brazil|    18|     MF|     FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
+-------+------+-------+---------+----------+-------------+--------------------+------+----+



Filtrar DF com 2 condições (AND / &)

In [22]:
df.filter((col('Selecao') == "Argentina") & (col('Altura') > 180) & (col('Peso') >= 85)).show()

+---------+------+-------+--------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|     Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+--------------+----------+-------------+--------------------+------+----+
|Argentina|     6|     DF|FAZIO Federico|17.03.1987|        FAZIO|       AS Roma (ITA)|   199|  85|
|Argentina|    12|     GK| ARMANI Franco|16.10.1986|       ARMANI|CA River Plate (ARG)|   189|  85|
|Argentina|     1|     GK| GUZMAN Nahuel|10.02.1986|       GUZMÁN|   Tigres UANL (MEX)|   192|  90|
+---------+------+-------+--------------+----------+-------------+--------------------+------+----+



In [23]:
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
only showing top 5 rows



Filtrtar com OR

In [24]:
df.filter((col('Nome_FIFA') == 'MESSI Lionel') | (col('Nome_FIFA') == 'SALVIO Eduardo') | (col('Altura') == 199)).show(5)

+---------+------+-------+----------------+----------+-------------+------------------+------+----+
|  Selecao|Numero|Posicao|       Nome_FIFA|Nascimento|Nome Camiseta|              Time|Altura|Peso|
+---------+------+-------+----------------+----------+-------------+------------------+------+----+
|Argentina|    18|     DF|  SALVIO Eduardo|13.07.1990|       SALVIO|  SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|    MESSI Lionel|24.06.1987|        MESSI|FC Barcelona (ESP)|   170|  72|
|Argentina|     6|     DF|  FAZIO Federico|17.03.1987|        FAZIO|     AS Roma (ITA)|   199|  85|
|  Belgium|     1|     GK|COURTOIS Thibaut|11.05.1992|     COURTOIS|  Chelsea FC (ENG)|   199|  91|
+---------+------+-------+----------------+----------+-------------+------------------+------+----+



Filtrar combinando & e |

In [25]:
df.filter((col('Selecao') == "Brazil") & (col('Posicao') == 'DF') | (col('Altura') == 199) & (col('Selecao') == 'Belgium')).show()

+-------+------+-------+----------------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|       Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+----------------+----------+-------------+--------------------+------+----+
|Belgium|     1|     GK|COURTOIS Thibaut|11.05.1992|     COURTOIS|    Chelsea FC (ENG)|   199|  91|
| Brazil|    22|     DF|          FAGNER|11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|     6|     DF|     FILIPE LUIS|09.08.1985|  FILIPE LUIS|Atletico Madrid (...|   182|  73|
| Brazil|    13|     DF|      MARQUINHOS|14.05.1994|   MARQUINHOS|Paris Saint-Germa...|   183|  75|
| Brazil|     3|     DF|         MIRANDA|07.09.1984|      MIRANDA|FC Internazionale...|   186|  78|
| Brazil|    14|     DF|          DANILO|15.07.1991|       DANILO|Manchester City F...|   184|  78|
| Brazil|     2|     DF|    THIAGO SILVA|22.09.1984|     T. SILVA|Paris Saint-Germa...|   183|  79|


Criar novas colunas (func lit)

In [26]:
df.withColumn('worldCup', lit('2018')).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|worldCup|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    2018|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|    2018|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|    2018|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|    2018|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|    2018|
+---------+------+-------+------------------+----------+-------------+--------------------+-----

Criar coluna condicional (substring)

In [27]:
df.withColumn('Sub', substring('selecao', 1, 3)).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Sub|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|Arg|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|Arg|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|Arg|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|Arg|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|Arg|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+
only showing top 5 rows



In [28]:
df = df.withColumn('Ano', substring('Nascimento', -4, 4))

In [29]:
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|1987|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
only showing top 5 

Criar coluna condicional (concat / concat_ws)

In [30]:
df.withColumn('Concat', concat('Selecao', 'Time')).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+--------------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|              Concat|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+--------------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992|ArgentinaAFC Ajax...|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996|ArgentinaCA Boca ...|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993|ArgentinaWest Ham...|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990|ArgentinaSL Benfi...|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)

In [31]:
df.withColumn('Concat', concat_ws(' - ', 'Selecao', 'Time')).show(5)


+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+--------------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|              Concat|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+--------------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992|Argentina - AFC A...|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996|Argentina - CA Bo...|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993|Argentina - West ...|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990|Argentina - SL Be...|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)

In [32]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: string (nullable = true)



Alterar tipo de coluna

In [33]:
df = df.withColumn('Ano', col('Ano').cast(IntegerType()))

In [34]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: integer (nullable = true)



Converter para string para data

In [35]:
mes = udf(lambda data: data.split('.')[1])

In [36]:
df = df.withColumn('Mes', mes('Nascimento'))

In [37]:
dia = udf(lambda data: data.split('.')[0])

In [38]:
df = df.withColumn('Dia', dia('Nascimento'))

In [46]:
df = df.withColumn('Data_Nascimento', concat_ws('-', 'Ano', 'Mes', 'Dia'))

In [47]:
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+---+---+---------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|Mes|Dia|Data_Nascimento|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+---+---+---------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992|  8| 31|      1992-8-31|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996|  1| 21|      1996-1-21|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993|  2| 15|      1993-2-15|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990|  7| 13|      1990-7-13|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI

In [41]:
df = df.withColumn('Mes', col('Mes').cast(IntegerType()))

In [42]:
df = df.withColumn('Dia', col('Dia').cast(IntegerType()))

In [48]:
df = df.withColumn('Data_Nascimento', col('Data_Nascimento').cast(DateType()))

In [49]:
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+---+---+---------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|Mes|Dia|Data_Nascimento|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+---+---+---------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992|  8| 31|     1992-08-31|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996|  1| 21|     1996-01-21|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993|  2| 15|     1993-02-15|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990|  7| 13|     1990-07-13|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI

In [50]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Mes: integer (nullable = true)
 |-- Dia: integer (nullable = true)
 |-- Data_Nascimento: date (nullable = true)

